Instalações e importações necessárias

In [1]:
!pip install pydantic
!pip install pydantic[email]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 8.2 MB/s eta 0:00:00


In [2]:
import enum
import hashlib
import re
from typing import Any

from pydantic import (
    BaseModel,
    EmailStr,
    Field,
    field_validator,
    model_validator,
    SecretStr,
    ValidationError,
)

Criação de expressões regulares para validação
A primeira garante que a senha tenha pelo menos oito caracteres, contendo letra maiúscula, minúscula e um número
Já a segunda garante que o nome tenha apenas letras e no mínimo 2 caracteres

In [3]:
VALID_PASSWORD_REGEX = re.compile(r"^(?=.*[a-z])(?=.*[A-Z])(?=.*\d).{8,}$")
VALID_NAME_REGEX = re.compile(r"^[a-zA-Z]{2,}$")

Criação da classe Role, porém agora, diferente do exemplo 1, os papéis tem valores específicos atribuídos (em potencias de 2)

In [4]:
class Role(enum.IntFlag):
    Author = 1
    Editor = 2
    Admin = 4
    SuperAdmin = 8

Criação da classe User, bem semelhante ao do exemplo 1, porém agora possui um field_validator para "name", que valida que o nome seja compatível com a expressão regular citada acima e lança um erro caso seja inválido

In [5]:
class User(BaseModel):
    name: str = Field(examples=["Arjan"])
    email: EmailStr = Field(
        examples=["user@arjancodes.com"],
        description="The email address of the user",
        frozen=True,
    )
    password: SecretStr = Field(
        examples=["Password123"], description="The password of the user"
    )
    role: Role = Field(
        default=None, description="The role of the user", examples=[1, 2, 4, 8]
    )

    @field_validator("name")
    @classmethod
    def validate_name(cls, v: str) -> str:
        if not VALID_NAME_REGEX.match(v):
            raise ValueError(
                "Name is invalid, must contain only letters and be at least 2 characters long"
            )
        return v

Há também o field_validator para "role", que aceita o papel como inteiro, string ou instância, usando um dicionário de operações para converter o valor do tipo Role correto e lança um erro se o papel for inválido

In [6]:
    @field_validator("role", mode="before")
    @classmethod
    def validate_role(cls, v: int | str | Role) -> Role:
        op = {int: lambda x: Role(x), str: lambda x: Role[x], Role: lambda x: x}
        try:
            return op[type(v)](v)
        except (KeyError, ValueError):
            raise ValueError(
                f"Role is invalid, please use one of the following: {', '.join([x.name for x in Role])}"
            )

model_validator roda antes do processo de validação, verificando se o nome e a senha estão presentes nos dados. Além de garantir que a senha não contenha o nome do usuário, transformando a senha em um hashsha-256 como medida de segurança

In [7]:
    @model_validator(mode="before")
    @classmethod
    def validate_user(cls, v: dict[str, Any]) -> dict[str, Any]:
        if "name" not in v or "password" not in v:
            raise ValueError("Name and password are required")
        if v["name"].casefold() in v["password"].casefold():
            raise ValueError("Password cannot contain name")
        if not VALID_PASSWORD_REGEX.match(v["password"]):
            raise ValueError(
                "Password is invalid, must contain 8 characters, 1 uppercase, 1 lowercase, 1 number"
            )
        v["password"] = hashlib.sha256(v["password"].encode()).hexdigest()
        return v

Criação da função validate e main, essa última contém a definição de vários cenários de testes, iterando sobre cada um e imprimindo o nome do teste e executando a validação

In [9]:
def validate(data: dict[str, Any]) -> None:
    try:
        user = User.model_validate(data)
        print(user)
    except ValidationError as e:
        print("User is invalid:")
        print(e)


def main() -> None:
    test_data = dict(
        good_data={
            "name": "Arjan",
            "email": "example@arjancodes.com",
            "password": "Password123",
            "role": "Admin",
        },
        bad_role={
            "name": "Arjan",
            "email": "example@arjancodes.com",
            "password": "Password123",
            "role": "Programmer",
        },
        bad_data={
            "name": "Arjan",
            "email": "bad email",
            "password": "bad password",
        },
        bad_name={
            "name": "Arjan<-_->",
            "email": "example@arjancodes.com",
            "password": "Password123",
        },
        duplicate={
            "name": "Arjan",
            "email": "example@arjancodes.com",
            "password": "Arjan123",
        },
        missing_data={
            "email": "<bad data>",
            "password": "<bad data>",
        },
    )

    for example_name, data in test_data.items():
        print(example_name)
        validate(data)
        print()

In [10]:
if __name__ == "__main__":
    main()

good_data
User is invalid:
1 validation error for User
role
  Input should be 1, 2, 4 or 8 [type=enum, input_value='Admin', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/enum

bad_role
User is invalid:
1 validation error for User
role
  Input should be 1, 2, 4 or 8 [type=enum, input_value='Programmer', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/enum

bad_data
User is invalid:
1 validation error for User
email
  value is not a valid email address: An email address must have an @-sign. [type=value_error, input_value='bad email', input_type=str]

bad_name
User is invalid:
1 validation error for User
name
  Value error, Name is invalid, must contain only letters and be at least 2 characters long [type=value_error, input_value='Arjan<-_->', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

duplicate
name='Arjan' email='example@arjancodes.com' password=SecretStr('*